# Momentos de HU - Base Four Shape


## Objetivo Geral:
Utilizando os códigos Matlab/Octave juntamente com a base de formas, mostre a acurácia para a tarefa. Utilize 60% para treinamento e 40% para teste. Selecione as 600 primeiras imagens para treino e a restante para teste.


## Objetivos Específicos:

1. Utilizando os códigos de Momentos Invariantes de HU ([código de exemplo](https://drive.matlab.com/sharing/03e51873-545c-4e56-8e5e-e52deb71bf79)). Existe outros códigos em Python, C, e outros
2. Utilize a base four shapes ([Base de Dados](https://www.dropbox.com/scl/fi/j23s8ck7brc1107b6jx3w/fourShapes_2.tar.gz?rlkey=hkvy84zqqukgmf833ae9i4x6a&dl=0));
3. Utilize o k-NN como classificador (neste momento só vamos usar o Classificador k-NN) - Pode usar pronto do Matlab ou do Scikit-learn.
4. Avalie com diferentes parâmetros (número de zonas horizontais e verticais) para os momentos invariantes de HU e apresente o melhor resultado.


In [1]:
!tar -xzf ./fourShapes_2.tar.gz

In [11]:
# Configurando o Ambiente
import glob

TRAINING_FILES = 600 # Quantidade de arquivos para treinamento

# Lista com o caminho das pastas
circlePath = "./1_circle/"
squarePath = "./2_square/"
starPath = "./3_star/"
trianglePath = "./4_triangle/"

In [15]:
# Função para listar os arquivos e separar os pirmeiros TRAINING_FILES como treino
def spread_training_test(folder_path):
  files = glob.glob(f"{folder_path}*.png")
  return (files[:TRAINING_FILES], files[TRAINING_FILES:])

# Separando arquivo de treino e de teste
circleTraingFilesList, circleTestFilesList = spread_training_test(circlePath)
squareTraingFilesList, squareTestFilesList = spread_training_test(squarePath)
starTraingFilesList, starTestFilesList = spread_training_test(starPath)
triangleTraingFilesList, triangleTestFilesList = spread_training_test(trianglePath)

In [19]:
from PIL import Image
import numpy as np

# Função para abrir os arquivos como double e normalizar a imagem preto e branca
def open_files(file_list):
  np_array_list = []
  for file in file_list:
    img = Image.open(file)
    img_array = np.array(img, dtype=np.float64) / 255.0
    np_array_list.append(img_array)
  return np.array(np_array_list)

# Abrindo arquivos de treino
X_train_circle = open_files(circleTraingFilesList)
X_train_square = open_files(squareTraingFilesList)
X_train_star = open_files(starTraingFilesList)
X_train_triangle = open_files(triangleTraingFilesList)

# Abrindo arquivos de teste
X_test_circle = open_files(circleTestFilesList)
X_test_square = open_files(squareTestFilesList)
X_test_star = open_files(starTestFilesList)
X_test_triangle = open_files(triangleTestFilesList)

In [22]:
# Juntar todos os treinos
X_train = np.concatenate([X_train_circle, X_train_square, X_train_star, X_train_triangle], axis=0)
X_test  = np.concatenate([X_test_circle, X_test_square, X_test_star, X_test_triangle], axis=0)

# Criar os rótulos (0=circle, 1=square, 2=star, 3=triangle)
y_train = np.concatenate([
    np.zeros(len(X_train_circle)),
    np.ones(len(X_train_square)),
    2*np.ones(len(X_train_star)),
    3*np.ones(len(X_train_triangle))
], axis=0)

y_test = np.concatenate([
    np.zeros(len(X_test_circle)),
    np.ones(len(X_test_square)),
    2*np.ones(len(X_test_star)),
    3*np.ones(len(X_test_triangle))
], axis=0)

In [23]:
from skimage.measure import moments_hu, moments

# Função para calcular momentos de Hu de todas as imagens
def extract_hu_features(images):
    features = []
    for img in images:
        m = moments(img)
        hu = moments_hu(m)
        features.append(hu)
    return np.array(features)

# Extrair features
X_train_features = extract_hu_features(X_train)
X_test_features  = extract_hu_features(X_test)

X_train_features shape: (2400, 7)


In [45]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Cria o classificador
knn = KNeighborsClassifier(n_neighbors=1)
# Resultados:
# Padrão (distância euclidiana), 1 vizinho me permitiu 0.764375 e 3 0.7775
# Manhattan, 1 vizinho me deu 76125 e 3 0.775

# Treina
knn.fit(X_train_features, y_train)

# Prediz
y_pred = knn.predict(X_test_features)

# Avalia
acc = accuracy_score(y_test, y_pred)
print("Acurácia do k-NN:", acc)

Acurácia do k-NN: 0.764375


In [57]:
# Funções de Hu Moments para calcular por zonas

# Função para dividir a imagem em zonas
def split_image(img, nx, ny):
    h, w = img.shape
    zones = []
    h_step = h // ny
    w_step = w // nx
    for i in range(ny):
        for j in range(nx):
            zone = img[i*h_step:(i+1)*h_step, j*w_step:(j+1)*w_step]
            zones.append(zone)
    return zones

# Função para calcular os momentos de hu em cada zona
def hu_features_zones(img, nx=2, ny=2):
    zones = split_image(img, nx, ny)
    features = []
    for z in zones:
        m = moments(z)
        hu = moments_hu(m)
        features.extend(hu)
    return np.array(features)

# Função para calcular os momentos de hu par auma lista de imagens
def extract_features_all(images, nx=2, ny=2):
    all_features = []
    for img in images:
        feat = hu_features_zones(img, nx, ny)
        all_features.append(feat)
    return np.array(all_features)

In [64]:
# Classificação com k-NN usando diferentes parâmetros
# (número de zonas horizontais e verticais) para os momentos invariantes de HU

best_acc = 0
best_params = (1,1)

for nx in range(1, 5):
    for ny in range(1, 5):
        # Extrai features
        X_train_f = extract_features_all(X_train, nx, ny)
        X_test_f  = extract_features_all(X_test, nx, ny)

        # Treina k-NN
        knn = KNeighborsClassifier(n_neighbors=3)
        knn.fit(X_train_f, y_train)
        y_pred = knn.predict(X_test_f)

        # Avalia
        acc = accuracy_score(y_test, y_pred)
        print(f"Acurácia {nx}x{ny} zonas: {acc:.4f}")

        # Guarda melhor resultado
        if acc > best_acc:
            best_acc = acc
            best_params = (nx, ny)

print(f"\nMelhor acurácia: {best_acc:.4f} com {best_params[0]}x{best_params[1]} zonas")

Acurácia 1x1 zonas: 0.7775
Acurácia 1x2 zonas: 0.8506
Acurácia 1x3 zonas: 0.9081
Acurácia 1x4 zonas: 0.9581
Acurácia 2x1 zonas: 0.8606
Acurácia 2x2 zonas: 0.9762
Acurácia 2x3 zonas: 0.9831
Acurácia 2x4 zonas: 0.9925
Acurácia 3x1 zonas: 0.9081
Acurácia 3x2 zonas: 0.9825
Acurácia 3x3 zonas: 0.9950
Acurácia 3x4 zonas: 0.9938
Acurácia 4x1 zonas: 0.9550
Acurácia 4x2 zonas: 0.9919
Acurácia 4x3 zonas: 0.9938
Acurácia 4x4 zonas: 0.9931

Melhor acurácia: 0.9950 com 3x3 zonas
